Read the full dataset and keep data of only one bookmaker and in a certain period of time.

* Input: "dataset_all_columns.csv"
* Output: "predictions_bookmaker.csv"

In [1]:
import pandas as pd

In [2]:
# Reading data.

df_read = pd.read_csv('./data/dataset_all_columns.csv', sep=';', parse_dates=['Date'], low_memory=False)
df_read

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,SP1,2000-09-09,Barcelona,Malaga,2,1,H,2,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SP1,2000-09-09,La Coruna,Ath Bilbao,2,0,H,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SP1,2000-09-09,Real Madrid,Valencia,2,1,H,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SP1,2000-09-09,Sociedad,Santander,2,2,D,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SP1,2000-09-09,Zaragoza,Espanol,1,2,A,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8243,SP1,2022-05-22,Granada,Espanol,0,0,D,0,0,D,...,2.29,-1.25,2.02,1.88,2.03,1.90,2.06,2.05,1.97,1.90
8244,SP1,2022-05-22,Osasuna,Mallorca,0,2,A,0,0,D,...,1.76,0.25,1.68,2.15,1.74,2.23,1.86,2.29,1.77,2.11
8245,SP1,2022-05-22,Barcelona,Villarreal,0,2,A,0,1,A,...,2.50,-0.50,2.07,1.83,2.08,1.85,2.09,2.00,1.97,1.91
8246,SP1,2022-05-22,Sevilla,Ath Bilbao,1,0,H,0,0,D,...,1.69,0.00,2.01,1.89,2.02,1.90,2.07,2.06,2.00,1.88


In [3]:
# Checking which bookmaker has more historical data.
# We are considering last 10 seasons: from 2012-08-18 to 2022-05-22.
# There are 380 matches per season, so 3800 data points would be ideal.

# It turns out that these are 100 % complete:
# B365 - Bet365
# BW - Bet&Win
# WH - William Hill

# Betting odds for weekend games are collected Friday afternoons, and on Tuesday afternoons for midweek games.

df_check = df_read.loc[ (df_read['Date']>='2012-08-18') & (df_read['Date']<='2022-05-22') ].copy()

df_check[['B365H','B365D','B365A','BSH','BSD','BSA','BWH','BWD','BWA',
          'GBH','GBD','GBA','IWH','IWD','IWA','LBH','LBD','LBA',
          'SOH','SOD','SOA','SBH','SBD','SBA','SJH','SJD','SJA',
          'SYH','SYD','SYA','VCH','VCD','VCA','WHH','WHD','WHA']].info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3800 entries, 4448 to 8247
Data columns (total 36 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   B365H   3800 non-null   float64
 1   B365D   3800 non-null   float64
 2   B365A   3800 non-null   float64
 3   BSH     380 non-null    float64
 4   BSD     380 non-null    float64
 5   BSA     380 non-null    float64
 6   BWH     3800 non-null   float64
 7   BWD     3800 non-null   float64
 8   BWA     3800 non-null   float64
 9   GBH     375 non-null    float64
 10  GBD     375 non-null    float64
 11  GBA     375 non-null    float64
 12  IWH     3794 non-null   float64
 13  IWD     3794 non-null   float64
 14  IWA     3794 non-null   float64
 15  LBH     2277 non-null   float64
 16  LBD     2277 non-null   float64
 17  LBA     2277 non-null   float64
 18  SOH     0 non-null      float64
 19  SOD     0 non-null      float64
 20  SOA     0 non-null      float64
 21  SBH     0 non-null      float64
 2

In [4]:
# Keeping data of only one bookmaker.

bookmaker = 'B365'

df = df_read.loc[ (df_read['Date']>='2012-08-18') & (df_read['Date']<='2022-05-22') ].copy()
df = df[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', bookmaker+'H', bookmaker+'D', bookmaker+'A']]

# Renaming columns.
df = df.rename(columns={
    'Date': 'date', 
    'HomeTeam': 'team_home', 
    'AwayTeam': 'team_away', 
    'FTHG': 'goals_home', 
    'FTAG': 'goals_away',
    bookmaker+'H': 'odds_home', 
    bookmaker+'D': 'odds_draw' , 
    bookmaker+'A': 'odds_away',
                    })

# Transforming odds in probabilities.
df[['odds_home', 'odds_draw', 'odds_away']] = round(1/df[['odds_home', 'odds_draw', 'odds_away']], 3)
df = df.rename(columns={
    'odds_home': 'probability_home', 
    'odds_draw': 'probability_draw' , 
    'odds_away': 'probability_away'
                    })

df

,date,team_home,team_away,goals_home,goals_away,probability_home,probability_draw,probability_away
4448,2012-08-18,Celta,Malaga,0,1,0.444,0.308,0.312
4449,2012-08-18,Mallorca,Espanol,2,1,0.500,0.303,0.263
4450,2012-08-18,Sevilla,Getafe,2,1,0.617,0.267,0.182
4451,2012-08-19,Ath Bilbao,Betis,3,5,0.546,0.286,0.238
4452,2012-08-19,Barcelona,Sociedad,5,1,0.909,0.111,0.038
...,...,...,...,...,...,...,...,...
8243,2022-05-22,Granada,Espanol,0,0,0.694,0.231,0.133
8244,2022-05-22,Osasuna,Mallorca,0,2,0.303,0.294,0.455
8245,2022-05-22,Barcelona,Villarreal,0,2,0.476,0.250,0.333
8246,2022-05-22,Sevilla,Ath Bilbao,1,0,0.400,0.303,0.348


In [5]:
# Checking categorical predictions and observations.

predictions = []
observations = []
is_true = []

for match in df.itertuples():
    
    # Categorical prediction.
    if match.probability_home > match.probability_draw and match.probability_home > match.probability_away:
        predictions.append('home')
    elif match.probability_away > match.probability_home and match.probability_away > match.probability_draw:
        predictions.append('away')
    elif match.probability_draw > match.probability_home and match.probability_draw > match.probability_away:
        predictions.append('draw')
    else:
        predictions.append('unknown')
        
    # Categorical observation.
    if match.goals_home > match.goals_away:
        observations.append('home')
    elif match.goals_away > match.goals_home:
        observations.append('away')
    elif match.goals_home == match.goals_away:
        observations.append('draw')
    else:
        observations.append('draw')
        
    # Check if prediction is right.
    if predictions[-1] == observations[-1]:
        is_true.append(1)
    else:
        is_true.append(0)

            
df['prediction'] = predictions
df['observation'] = observations
df['is_true'] = is_true
df

,date,team_home,team_away,goals_home,goals_away,probability_home,probability_draw,probability_away,prediction,observation,is_true
4448,2012-08-18,Celta,Malaga,0,1,0.444,0.308,0.312,home,away,0
4449,2012-08-18,Mallorca,Espanol,2,1,0.500,0.303,0.263,home,home,1
4450,2012-08-18,Sevilla,Getafe,2,1,0.617,0.267,0.182,home,home,1
4451,2012-08-19,Ath Bilbao,Betis,3,5,0.546,0.286,0.238,home,away,0
4452,2012-08-19,Barcelona,Sociedad,5,1,0.909,0.111,0.038,home,home,1
...,...,...,...,...,...,...,...,...,...,...,...
8243,2022-05-22,Granada,Espanol,0,0,0.694,0.231,0.133,home,draw,0
8244,2022-05-22,Osasuna,Mallorca,0,2,0.303,0.294,0.455,away,away,1
8245,2022-05-22,Barcelona,Villarreal,0,2,0.476,0.250,0.333,home,away,0
8246,2022-05-22,Sevilla,Ath Bilbao,1,0,0.400,0.303,0.348,home,home,1


In [6]:
# Bookmaker's accuracy.

df['is_true'].mean()

0.5360526315789473

In [7]:
# Saving as CSV.

df.to_csv('data/predictions_bookmaker.csv', index=False, encoding='UTF-8', sep=';')